In [4]:
import torch
import torchvision
import torchvision.transforms as transforms

In [2]:
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time

In [3]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32, shuffle=False)

100%|███████████████████████████████████████████████████████████████| 170498071/170498071 [01:15<00:00, 2268626.97it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [31]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()

        self.features = nn.Sequential(
            # First Conv Block
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 32x32 → 16x16

            # Second Conv Block
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 16x16 → 8x8

            # Third Conv Block
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2),  # 8x8 → 4x4
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 4 * 4, 256),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(256, 10)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)  # flatten
        x = self.classifier(x)
        return x


model = CNN()

In [34]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
start_time = time.time()

for epoch in range(20):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        epoch_time = time.time()
    print(f"Epoch {epoch + 1}, Loss: {running_loss:.3f} completed in {epoch_time - start_time:.2f} seconds")


total_time = time.time() - start_time
print(f"\nTotal Training Time: {total_time:.2f} seconds")

Epoch 1, Loss: 135.293 completed in 36.25 seconds
Epoch 2, Loss: 92.850 completed in 72.48 seconds
Epoch 3, Loss: 70.058 completed in 108.74 seconds
Epoch 4, Loss: 52.545 completed in 145.01 seconds
Epoch 5, Loss: 39.188 completed in 181.35 seconds
Epoch 6, Loss: 28.500 completed in 217.60 seconds
Epoch 7, Loss: 19.905 completed in 253.86 seconds
Epoch 8, Loss: 14.585 completed in 290.31 seconds
Epoch 9, Loss: 9.261 completed in 326.83 seconds
Epoch 10, Loss: 7.275 completed in 363.27 seconds
Epoch 11, Loss: 4.056 completed in 399.96 seconds
Epoch 12, Loss: 2.903 completed in 437.06 seconds
Epoch 13, Loss: 3.241 completed in 474.50 seconds
Epoch 14, Loss: 1.923 completed in 512.42 seconds
Epoch 15, Loss: 3.653 completed in 550.02 seconds
Epoch 16, Loss: 0.792 completed in 588.18 seconds
Epoch 17, Loss: 0.579 completed in 626.66 seconds
Epoch 18, Loss: 2.451 completed in 665.57 seconds
Epoch 19, Loss: 1.042 completed in 704.23 seconds
Epoch 20, Loss: 0.340 completed in 743.43 seconds

T

In [41]:
torch.save(model.state_dict(), 'model.pth')

In [35]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in testlo!ader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")



Test Accuracy: 80.94%


# 1. To Create Directory structure to store inference.py and weights
# 2. To Compress them into tarball to upload into s3 bucket

In [37]:
!mkdir model.tar.gz

In [39]:
!mkdir model

In [40]:
!mkdir model/code

In [48]:
!tar -czvf model.tar.gz -C model model.pth code/

model.pth
code/
code/.ipynb_checkpoints/
code/.ipynb_checkpoints/inference-checkpoint.py
code/inference.py
